In [1]:
import gzip
import csv
import json
import datetime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
def hex_to_rgb(c):
    return np.array([int(c[1:3], 16), int(c[3:5], 16), int(c[5:7], 16)], dtype=np.uint8)

def add_color(c):
    global hex_to_id, id_to_hex
    if c not in hex_to_id:
        hex_to_id[c] = np.int8(len(id_to_hex))
        id_to_hex += [c]

def ids_to_pic(ids):
    id_to_np = np.array([hex_to_rgb(c) for c in id_to_hex], dtype=np.uint8)
    pic = np.zeros((2000, 2000, 3), dtype=np.uint8)
    for y in range(2000):
        for x in range(2000):
            pic[y, x] = id_to_np[ids[y, x]]
    return pic

def draw_pic(pic):
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(pic)
    plt.show()


In [5]:
hex_to_id = {'#FFFFFF': np.int8(0)}
id_to_hex = ['#FFFFFF']

place_grid = np.zeros((2000, 2000), dtype=np.int8)

mod_rows = []

In [6]:
# First pass
# Get all colors, number of rows, etc.

with gzip.open('data/2022_place_canvas_history.csv.gzip', mode='rt', newline='') as f:
    f.readline() # skip header
    reader = csv.reader(f)
    start_date = datetime.datetime(2030, 1, 1).timestamp()
    max_i = 0
    num_mod = 0
    for i, row in enumerate(reader):
        try:
            timestamp = datetime.datetime.strptime(row[0], r'%Y-%m-%d %H:%M:%S.%f %Z').timestamp()
        except ValueError:
            timestamp = datetime.datetime.strptime(row[0], r'%Y-%m-%d %H:%M:%S %Z').timestamp()
        if timestamp < start_date:
            start_date = timestamp

        add_color(row[2])

        location = tuple([int(x) for x in row[3].split(',')][::-1])
        if len(location) > 2:
            mod_rows += [row]

        max_i = i

        if i % 1e6 == 0:
            print(i)
    print(max_i)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
111000

In [18]:
print('start timestamp:', start_date)
print('total rows:', max_i + 1)
print('mod actions:', len(mod_rows))
print('colors:', list(reversed(sorted(id_to_hex))))

with open('stats.json', 'w') as f:
    json.dump({
        'start_time': start_date,
        'rows': max_i + 1,
        'censors': mod_rows,
        'colors': list(reversed(sorted(id_to_hex))),
    }, f)

start timestamp: 1648831450.315
total rows: 160353104
mod actions: 19
colors: ['#FFFFFF', '#FFF8B8', '#FFD635', '#FFB470', '#FFA800', '#FF99AA', '#FF4500', '#FF3881', '#E4ABFF', '#DE107F', '#D4D7D9', '#BE0039', '#B44AC0', '#9C6926', '#94B3FF', '#898D90', '#811E9F', '#7EED56', '#6D482F', '#6D001A', '#6A5CFF', '#51E9F4', '#515252', '#493AC1', '#3690EA', '#2450A4', '#00CCC0', '#00CC78', '#00A368', '#009EAA', '#00756F', '#000000']


In [22]:
with open('stats.json', 'r') as f:
    stats = json.load(f)

n_rows = stats['rows']
colors = stats['colors']
color_ids = {c: np.uint32(i) for i, c in enumerate(colors)}

all_rows = np.zeros((n_rows, 6), dtype=np.uint32)

In [39]:
with gzip.open('data/2022_place_canvas_history.csv.gzip', mode='rt', newline='') as f:
    f.readline()
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        try:
            timestamp = datetime.datetime.strptime(row[0], r'%Y-%m-%d %H:%M:%S.%f %Z').timestamp()
        except ValueError:
            timestamp = datetime.datetime.strptime(row[0], r'%Y-%m-%d %H:%M:%S %Z').timestamp()
        
        all_rows[i, 0] = np.uint32((timestamp - start_date) * 1000)
        all_rows[i, 1] = color_ids[row[2]]

        location = [int(x) for x in row[3].split(',')]
        all_rows[i, 2] = location[0]
        all_rows[i, 3] = location[1]

        if len(location) > 2:
            all_rows[i, 4] = location[2]
            all_rows[i, 5] = location[3]

        if i % 1e6 == 0:
            print(int(i / 1e6))


0


In [52]:
with gzip.open('data/2022_place_canvas_history.csv.gzip', mode='rt', newline='') as f:
    for i in range(20):
        print(f.readline(), end='')

timestamp,user_id,pixel_color,coordinate
2022-04-04 00:53:51.577 UTC,ovTZk4GyTS1mDQnTbV+vDOCu1f+u6w+CkIZ6445vD4XN8alFy/6GtNkYp5MSic6Tjo/fBCCGe6oZKMAN3rEZHw==,#00CCC0,"826,1048"
2022-04-04 00:53:53.758 UTC,6NSgFa1CvIPly1VniNhlbrmoN3vgDFbMSKqh+c4TTfrr3dMib91oUWONX96g5PPcioIxedF24ldNOu/g5yqDrg==,#94B3FF,"583,1031"
2022-04-04 00:53:54.685 UTC,O5Oityp3Z3owzTuwM9XnMggpLcqKEumsOMKGhRiDTTImWbNLhLKmLI4gn1QPbaABqZqmFC/OmE/O732n39dGIQ==,#6A5CFF,"1873,558"
2022-04-04 00:54:57.541 UTC,tc273UiqS0wKa6VwiOs/iz/t4LyPYrhL2Q347awn11IQQELrEzZBCmGe28NWM+O1IdfH4CieCpEnE5sHecW9Ow==,#009EAA,"1627,255"
2022-04-04 00:55:16.307 UTC,OOWsU/HLb4UUkQwclDeXFtsJTOXMlAdNHiRpFA1Qk+SxUrJE7lpGFevfV9w+zImFimmNANlDdfN3kluz69M9MQ==,#94B3FF,"49,1478"
2022-04-04 00:55:20.64 UTC,A0HdtcPvI7ipKivvXNVZDa3gkcjGXFjNxF5tca5QXazENCGVR8d3j65QVgnaVIgvGbdtiGuSvRs1rAj2f1oMAQ==,#E4ABFF,"408,1863"
2022-04-04 00:55:34.898 UTC,1U4LPuB22P6Yf7eRhKz6zU1dFMK5wXIzsNVPUNhP7eHIwGuMfnDz/hDh/8QhDug6qqsKXHYvgH9L5FnWNMGm3A==,#94B3FF,"111,1582"
2022-04-0